# Collaboration and Competition

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the third project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) program.

### 1. Start the Environment

We begin by importing the necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [9]:
from unityagents import UnityEnvironment
import numpy as np
import torch
import os
from code import Agent, Config

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Tennis.app"`
- **Windows** (x86): `"path/to/Tennis_Windows_x86/Tennis.exe"`
- **Windows** (x86_64): `"path/to/Tennis_Windows_x86_64/Tennis.exe"`
- **Linux** (x86): `"path/to/Tennis_Linux/Tennis.x86"`
- **Linux** (x86_64): `"path/to/Tennis_Linux/Tennis.x86_64"`
- **Linux** (x86, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86"`
- **Linux** (x86_64, headless): `"path/to/Tennis_Linux_NoVis/Tennis.x86_64"`

For instance, if you are using a Mac, then you downloaded `Tennis.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Tennis.app")
```

In [3]:
if Config.SELECTED_ENV == 'tennis':
    ENV_PATH = Config.TENNIS_ENV_PATH
    CHECKPOINT_PATH = Config.CHECKPOINT_TENNIS_PATH
    MODEL_PATH = Config.MODEL_TENNIS_PATH

env = UnityEnvironment(file_name=ENV_PATH)

UnityTimeOutException: The Unity environment took too long to respond. Make sure that :
	 The environment does not need user interaction to launch
	 The Academy and the External Brain(s) are attached to objects in the Scene
	 The environment and the Python interface have compatible versions.

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])
agent = Agent(num_agents=num_agents, state_size=state_size, action_size=action_size, random_seed=10)

### 3. Use the learned DDPG (Learned using **DDPG.py **)


In [12]:
agent.actor_local.load_state_dict(torch.load(os.path.join(MODEL_PATH, 'checkpoint_actor.pth')))
agent.critic_local.load_state_dict(torch.load(os.path.join(MODEL_PATH, 'checkpoint_critic.pth')))

env_info = env.reset(train_mode=False)[brain_name]
for t in range(200):
    states = env_info.vector_observations
    actions = agent.act(states, add_noise=False)
    env_info = env.step(actions)[brain_name]
    dones = env_info.local_done
    if np.any(dones):
        break 

RuntimeError: Error(s) in loading state_dict for Actor:
	Unexpected key(s) in state_dict: "bn1.weight", "bn1.bias", "bn1.running_mean", "bn1.running_var". 
	While copying the parameter named "fc1.weight", whose dimensions in the model are torch.Size([256, 8]) and whose dimensions in the checkpoint are torch.Size([256, 24]).
	While copying the parameter named "fc2.weight", whose dimensions in the model are torch.Size([128, 256]) and whose dimensions in the checkpoint are torch.Size([256, 256]).
	While copying the parameter named "fc2.bias", whose dimensions in the model are torch.Size([128]) and whose dimensions in the checkpoint are torch.Size([256]).
	While copying the parameter named "fc3.weight", whose dimensions in the model are torch.Size([2, 128]) and whose dimensions in the checkpoint are torch.Size([2, 256]).

When finished, you can close the environment.

In [ ]:
env.close()